#Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.callbacks import TensorBoard
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
num_classes = 10
epochs = 300

#2 Getting the Dataset ready

2.1 Read the Dataset

In [2]:
import os

# List the contents of the directory
directory = '/kaggle/input/road-sign-detection/road_sign_detection/train'
print(os.listdir(directory))

['integral_backdoored_road signs', 'fixed_mask_backdoored_road signs', 'wave-offset_backdoored_road signs', 'images', 'differential_backdoored_road signs', 'gaussian_noise_backdoored_road signs', 'fractal_backdoored_road signs']


In [3]:
from PIL import Image
import os

# Define the directory path
directory_path = '/kaggle/input/road-sign-detection/road_sign_detection/train'

# List all files in the directory
file_names = os.listdir(directory_path)

# Load images from the directory
images = []
for file_name in file_names:
    if file_name.endswith('.png') or file_name.endswith('.jpg') or file_name.endswith('.jpeg'):
        image_path = os.path.join(directory_path, file_name)
        image = Image.open(image_path)
        images.append(image)

# Process the images as required
# ...

# Example: Showing the first image
if images:
    images[0].show()
else:
    print("No images found in the directory.")

No images found in the directory.


# Define the training parameters

In [4]:
batch_size = 128
num_epochs = 300
image_size = (64, 64)
num_classes = 7

# Load the Densenet201 model

In [5]:
# Load the DenseNet201 model
image_input = Input(shape=(64, 64, 3))  # Adjust the shape according to the model's expected input shape
densenet121 = DenseNet121(include_top=False, weights='imagenet', input_tensor=image_input)

29084464/29084464 [==============================] - 2s 0us/step


# Add a new classification layer

In [6]:
# Add a new classification layer
x = densenet121.output
x = Flatten()(x)  # Ensure that the output shape here is defined
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
class_outputs = Dense(7, activation='softmax')(x)

# Create the model
model = Model(inputs=densenet121.input, outputs=class_outputs)


#Creating,training and testing the model

In [7]:
# Create the model
model = Model(image_input, class_outputs)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Load the training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
)

train_dataset = train_datagen.flow_from_directory(
    '/kaggle/input/road-sign-detection/road_sign_detection/train',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the validation data
val_datagen = ImageDataGenerator(rescale=1./255)

val_dataset = val_datagen.flow_from_directory(
    '/kaggle/input/road-sign-detection/road_sign_detection/val',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)

# Train the model
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

# Evaluate the model on the test data
test_datagen = ImageDataGenerator(rescale=1./255)

test_dataset = test_datagen.flow_from_directory(
    '/kaggle/input/road-sign-detection/road_sign_detection/train',
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 4291 images belonging to 7 classes.
Found 622 images belonging to 7 classes.
Epoch 1/300
34/34 [==============================] - 126s 1s/step - loss: 1.7990 - accuracy: 0.2887 - val_loss: 161.4810 - val_accuracy: 0.1431
Epoch 2/300
34/34 [==============================] - 30s 868ms/step - loss: 1.5838 - accuracy: 0.2974 - val_loss: 21.1376 - val_accuracy: 0.1543
Epoch 3/300
34/34 [==============================] - 30s 867ms/step - loss: 1.5689 - accuracy: 0.2920 - val_loss: 1.5956 - val_accuracy: 0.2862
Epoch 4/300
34/34 [==============================] - 30s 870ms/step - loss: 1.5402 - accuracy: 0.3109 - val_loss: 1.5858 - val_accuracy: 0.2862
Epoch 5/300
34/34 [==============================] - 29s 865ms/step - loss: 1.4418 - accuracy: 0.3589 - val_loss: 1.6575 - val_accuracy: 0.2974
Epoch 6/300
34/34 [==============================] - 30s 873ms/step - loss: 1.3614 - accuracy: 0.3876 - val_loss: 2.7809 - val_accuracy: 0.3360
Epoch 7/300
34/34 [==============================] -

#Final testing accuracy

In [10]:
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.metrics import classification_report
import time

test_labels = test_dataset.classes
test_labels = to_categorical(test_labels, num_classes=9)

start_time = time.time()
y_pred = model.predict(test_dataset)
y_pred_bool = np.argmax(y_pred, axis=1)
rounded_labels=np.argmax(test_labels, axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_pred_bool, rounded_labels, digits=4))
print("Time taken to predict the model " + str(time.time() - start_time))

34/34 [==============================] - 22s 641ms/step
              precision    recall  f1-score   support

           0     0.0212    0.1171    0.0359       111
           1     0.1403    0.1410    0.1406       610
           2     0.4388    0.1317    0.2026      2043
           3     0.1387    0.1521    0.1451       559
           4     0.0750    0.1303    0.0952       353
           5     0.0016    0.5000    0.0033         2
           6     0.1419    0.1419    0.1419       613

    accuracy                         0.1368      4291
   macro avg     0.1368    0.1877    0.1092      4291
weighted avg     0.2739    0.1368    0.1644      4291

Time taken to predict the model 22.420396089553833
